In [0]:
df = spark.read.format("csv").option("header" , True).load("dbfs:/FileStore/shared_uploads/athiyastudies@gmail.com/original.csv")

In [0]:
df = df.withColumn("newSal",df.Salary.substr(2,100).cast("float"))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|  newSal|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199| 62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266

In [0]:
import pyspark.sql.functions as F

In [0]:
df1 = df.groupBy('gender').agg(F.avg('newSal').alias("Avg Sal"))

In [0]:
df1.show()

+------+-----------------+
|gender|          Avg Sal|
+------+-----------------+
|Female|55618.94298820185|
|  Male|55361.09385573019|
+------+-----------------+



In [0]:
from pyspark.sql.functions import lit,when,col

In [0]:
df2 = df.withColumn("FemaleSal",when(df.gender=='Female',df.newSal).otherwise(lit(0)))

In [0]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+---------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|  newSal|FemaleSal|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+---------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|57438.18| 57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199| 62846.6|  62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|57576.52| 57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|61489.23|      0.0|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|63863.09|      0.0|
|  6|   

In [0]:
df = df2.withColumn("MaleSal",when(df2.gender=='Male',df2.newSal).otherwise(lit(0)))

In [0]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+---------+--------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|  newSal|FemaleSal| MaleSal|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+---------+--------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|57438.18| 57438.18|     0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199| 62846.6|  62846.6|     0.0|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|57576.52| 57576.52|     0.0|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|61489.23|      0.0|61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sale

In [0]:
df1 = df.groupBy("JobTitle").agg(F.avg("FemaleSal").alias("Female_avg_sal"),F.avg("MaleSal").alias("Male_avg_sal"))

In [0]:
df1.show()

+--------------------+------------------+------------------+
|            JobTitle|    Female_avg_sal|      Male_avg_sal|
+--------------------+------------------+------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|
|   Media Manager III|29586.436197916668|17381.920572916668|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|
|       Geologist III|      31749.046875|    12830.75390625|
|        Geologist II|               0.0|   43293.865234375|
|Database Administ...|               0.0|     52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625|
|Software Engineer II|               0.0|      74782.640625|
|       Accountant IV|   82732.248046875|               0.0|
|    Product Engineer|    41825.48359375|       20464.94375|
|Software Test Eng...|  32218.6083984375|   27122.462890625|
|Safety Technician...|               0.0|   29421.529296875|
|    Junior Executive|15

In [0]:
df3 = df1.withColumn("delta",df1.Female_avg_sal-df1.Male_avg_sal)

In [0]:
df3.show()

+--------------------+------------------+------------------+-------------------+
|            JobTitle|    Female_avg_sal|      Male_avg_sal|              delta|
+--------------------+------------------+------------------+-------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III|29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|      31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|               0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|               0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|               0.0|      74782.640625|      -74782.640625|
|       Accountant IV|   827

In [0]:
avgCity = df.groupBy('City').agg(F.avg('newSal').alias('CittyAvg'))

In [0]:
highavgCity = avgCity.sort(col('CittyAvg').desc())

In [0]:
highavgCity.show()

+-----------------+-------------+
|             City|     CittyAvg|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows

